In [4]:
%use ktor-client, klaxon



Next, we'll define the API key and construct a function to fetch actor data. Replace `"YOUR_IMDB_API_KEY"` with your actual API key.


In [12]:
import io.ktor.client.*
import io.ktor.client.call.*
import io.ktor.client.request.*
import io.ktor.client.statement.*
import io.ktor.client.plugins.contentnegotiation.*
import io.ktor.serialization.kotlinx.json.*
import kotlinx.serialization.Serializable
import kotlinx.serialization.json.Json
import kotlinx.coroutines.runBlocking

val rapidApiKey = "dcc33b3befmshb7014447c7119e0p19d1b6jsn5baebab2d993"
val rapidApiHost = "imdb8.p.rapidapi.com"
val imdbSearchUrl = "https://imdb8.p.rapidapi.com/auto-complete"

// Data class to parse only what we need
@Serializable
data class Actor(
    val id: String,
    val l: String,     // label/name
    val s: String? = null // summary
)

@Serializable
data class ActorSearchResponse(
    val d: List<Actor> = emptyList()
)

suspend fun searchActors(name: String): ActorSearchResponse? {
    val client = HttpClient {
        install(ContentNegotiation) {
            json(Json { ignoreUnknownKeys = true }) // <-- important fix here
        }
    }

    return try {
        client.get(imdbSearchUrl) {
            headers {
                append("X-RapidAPI-Key", rapidApiKey)
                append("X-RapidAPI-Host", rapidApiHost)
            }
            parameter("q", name)
        }.body()
    } catch (e: Exception) {
        println("Error: ${e.message}")
        null
    } finally {
        client.close()
    }
}



In [13]:
// Usage
val actorName = "Tom Hanks"
val result = runBlocking { searchActors(actorName) }

println("Results for '$actorName':")
result?.d?.forEach {
    println("ID: ${it.id}, Name: ${it.l}, Info: ${it.s}")
}

Results for 'Tom Hanks':
ID: nm0000158, Name: Tom Hanks, Info: Producer, Cast Away (2000)
ID: tt26688393, Name: Tom Hanks: The Nomad, Info: Tom Hanks, Josie Ellis
ID: tt36674352, Name: Untitled Greyhound Sequel Tom Hanks, Info: Tom Hanks, Stephen Graham
ID: nm0451148, Name: Aamir Khan, Info: Actor, Dangal (2016)
ID: tt11885402, Name: Dan Aykroyd and Tom Hanks: City of Crime, Info: Dan Aykroyd, Tom Hanks
ID: nm16751374, Name: Tom Hanks Jr., Info: Actor, The Reef 2: High Tide (2012)
ID: tt6182460, Name: Tom Hanks: A League of His Own, Info: Victor Garber, Tom Hanks
ID: tt36726135, Name: The Moonwalkers: A Journey with Tom Hanks, Info: Tom Hanks
